In [6]:
#Basically I have to extract the right bedroom word....
from bs4 import BeautifulSoup

In [7]:
filePath = r"D:\propertyimages\webImages\531 W. Street Road - Gable _ County Builders Inc.html"
# filePath = r"D:\propertyimages\webImages\1130 - Quality Design Homes.html"
# filePath = r"D:\propertyimages\webImages\1272 Bentley Rd. [Contract Pending] - Gateway Construction.html"
# filePath = r"D:\propertyimages\webImages\1382 Ashwood Lane, Victor, NY 14564 (MLS # R1389827) _ Morrell Builders.html"
# filePath = r"D:\propertyimages\webImages\1704 N. Harrison Street.html"
# filePath = r"D:\propertyimages\webImages\3308 Club Court _ New Homes in Naperville IL _ Eastman Properties.html"
# filePath = r"D:\propertyimages\webImages\7138 Cassidy Court #Lot 219, Victor, NY 14564 (MLS # R1389188) _ Morrell Builders.html"
# filePath = r"D:\propertyimages\webImages\Atlantic - Schmidt Homes, LLC.html"
# filePath = r"D:\propertyimages\webImages\CAREFREE LIVING IN THE ENCLAVE AT AUGUSTA GREEN - Spencer Homes, LLC.html"
# filePath = r"D:\propertyimages\webImages\Chelsea Park.html"
# filePath = r"D:\propertyimages\webImages\Francis ADU 896 _ Skagit Design Homes.html"
# filePath = r"D:\propertyimages\webImages\Home _ Pate Jones Construction.html"
# filePath = r"D:\propertyimages\webImages\Home Plans - Severyn Development.html"
# filePath = r"D:\propertyimages\webImages\House Plan 1430 - Gough Homes.html"
# filePath = r"D:\propertyimages\webImages\Huntington - Newport Homes.html"
# filePath = r"D:\propertyimages\webImages\Lake Winnebago _ Pfeifer Homes Inc. _ Lee's Summit Kansas City.html"
# filePath = r"D:\propertyimages\webImages\McCabe Floor Plan _ Heartland Builders.html"
# filePath = r"D:\propertyimages\webImages\Riviera _ Pfeifer Homes Inc. _ Lee's Summit kansas City.html"
# filePath = r"D:\propertyimages\webImages\Saddle Ridge - Gateway Construction.html"
# filePath = r"D:\propertyimages\webImages\THE DAWSON - Colorado Springs Custom Homes by Jayden Homes.html"
# filePath = r"D:\propertyimages\webImages\The Westbrook _ Inspired Homes.html"
# filePath = r""

In [8]:
soup = ''
with open(filePath, encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

In [9]:
all_tags = soup.body.find_all()

req_tags = []

ignored_tags = ['script', 'style', 'wix-image']

main_tags = []

for tag in all_tags:
    if(tag.name not in ignored_tags):
        req_tags.append(tag)


for tag in req_tags:
    if(len(list(tag.descendants)) == 1):
        main_tags.append(tag)

keyword_tags = []

for tag in main_tags:
    content = tag.text.strip()
    if(content):
        content = content.lower()
        if('bed' in content):
            keyword_tags.append(tag)

if(len(keyword_tags) == 0):
    for tag in main_tags:
        content = tag.text.strip()
        if(content):
            content = content.lower()
            if('bath' in content):
                keyword_tags.append(tag)

for x in keyword_tags:
    print(x, '\n')


<p class="page-copy ng-binding ng-scope" ng-bind-html="home.description" ng-if="home.description">Our Gable End Model features 9' ceilings on the first and second floors with an open and exciting floor plan.  42" Kitchen cabinets with crown molding, granite countertops and built-in pantry featuring a classic sliding barn door.  Breakfast area leads out to a 10 x 10 Timbertech deck.  Spacious En Suite has tray ceiling with a Juliette balcony, luxury bath featuring a 5' shower and double bowl vanity with granite top.  Oversized 1-car garage with double parking pad. Finish Recreation room for entertaining.</p> 



In [10]:
src_list = []

numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

for tag in keyword_tags:
    content = tag.text.strip()
    flag = 0
    for number in numbers:
        if(number in content):
            flag = 1
            break
    if(flag == 0 and tag.next_sibling):
        if(isinstance(tag.next_sibling, str)):
            for number in numbers:
                if(number in tag.next_sibling):
                    flag = 1
                    break
    if(flag == 0 and tag.previous_sibling):
        if(isinstance(tag.next_sibling, str)):
            for number in numbers:
                if(number in tag.next_sibling):
                    flag = 1
                    break
    if(flag == 0 and tag.findNext()):
        if(isinstance(tag.findNext(), str)):
            for number in numbers:
                if(number in tag.findNext()):
                    flag = 1
                    break
        else:
            tempContent = tag.findNext().text.strip()
            if(tempContent):
                for number in numbers:
                    if(number in tempContent):
                        flag = 1
                        break
    if(flag == 0 and tag.findPrevious()):
        if(isinstance(tag.findPrevious(), str)):
            for number in numbers:
                if(number in tag.findPrevious()):
                    flag = 1
                    break
        else:
            tempContent = tag.findPrevious().text.strip()
            if(tempContent):
                for number in numbers:
                    if(number in tempContent):
                        flag = 1
                        break


    ignored_words = ['icon', 'facebook', 'twitter', 'instagram', 'logo', 'map']
    if(flag):
        print(flag)
        img_list = []
        parent = ''
        store = tag
        while(len(img_list) == 0):
            x = store.previous_sibling
            while(x):
                if(isinstance(x, str) == False):
                    images = x.find_all('img')
                    if(len(images) != 0):
                        for img in images:
                            marker = 0
                            for itr in ignored_words:
                                if (itr in img.get('src').lower()):
                                    print('Hello')
                                    marker = 1
                                    break
                            if(marker == 0):
                                img_list.append(img)
                                    
                x = x.previous_sibling

            store = store.parent
            images = store.find_all('img')
            if(len(images) != 0):
                for img in images:
                    img_list.append(img)
        if(len(img_list) != 0):
            for x in img_list:
                src_list.append(x.get('src'))
        break


1


In [11]:
src_list = list(set(src_list))
len(src_list)

36

In [12]:
for source in src_list:
    print(source)

https://dlqxt4mfnxo6k.cloudfront.net/countybuildersinc.com/aHR0cHM6Ly9zMy5hbWF6b25hd3MuY29tL2Jhc2UtYWRtaW4yL2NvdW50eWJ1aWxkZXJzaW5jLmNvbS83MTcxODg5NzIwNDg0NjFfZHNjXzAzNjkuanBn/exact/600/400
https://dlqxt4mfnxo6k.cloudfront.net/countybuildersinc.com/aHR0cHM6Ly9zMy5hbWF6b25hd3MuY29tL2Jhc2UtYWRtaW4yL2NvdW50eWJ1aWxkZXJzaW5jLmNvbS8yMDI4MDc4ODY1MjU5ODhfZHNjXzAzMTguanBn/exact/600/400
https://dlqxt4mfnxo6k.cloudfront.net/countybuildersinc.com/aHR0cHM6Ly9zMy5hbWF6b25hd3MuY29tL2Jhc2UtYWRtaW4yL2NvdW50eWJ1aWxkZXJzaW5jLmNvbS85MTA3OTU0NDY5NTA5NDJfZHNjXzAzMjIuanBn/exact/600/400
https://c.basemaps.cartocdn.com/light_all/13/2390/3096.png
/images/map-markers/map-pin.svg
/images/email2.svg
https://dlqxt4mfnxo6k.cloudfront.net/countybuildersinc.com/aHR0cHM6Ly9zMy5hbWF6b25hd3MuY29tL2Jhc2UtYWRtaW4yL2NvdW50eWJ1aWxkZXJzaW5jLmNvbS81MDM1NDc5NTUzMDQzOF9kc2NfMDMwMy5qcGc=/exact/600/400
https://b.basemaps.cartocdn.com/light_all/13/2391/3097.png
https://a.basemaps.cartocdn.com/light_all/13/2387/3097.png
https://dlqx